In [1]:
from bs4 import BeautifulSoup, Tag
import zipfile
import olefile

In [2]:
file = 'examples/with_attachments.docx'

In [3]:
document = zipfile.ZipFile(file)
bs = BeautifulSoup(document.read('word/document.xml'), 'xml')

In [4]:
print(bs.prettify())

<?xml version="1.0" encoding="utf-8"?>
<w:document mc:Ignorable="w14 w15 w16se wp14" xmlns:cx="http://schemas.microsoft.com/office/drawing/2014/chartex" xmlns:m="http://schemas.openxmlformats.org/officeDocument/2006/math" xmlns:mc="http://schemas.openxmlformats.org/markup-compatibility/2006" xmlns:o="urn:schemas-microsoft-com:office:office" xmlns:r="http://schemas.openxmlformats.org/officeDocument/2006/relationships" xmlns:v="urn:schemas-microsoft-com:vml" xmlns:w="http://schemas.openxmlformats.org/wordprocessingml/2006/main" xmlns:w10="urn:schemas-microsoft-com:office:word" xmlns:w14="http://schemas.microsoft.com/office/word/2010/wordml" xmlns:w15="http://schemas.microsoft.com/office/word/2012/wordml" xmlns:w16se="http://schemas.microsoft.com/office/word/2015/wordml/symex" xmlns:wne="http://schemas.microsoft.com/office/word/2006/wordml" xmlns:wp="http://schemas.openxmlformats.org/drawingml/2006/wordprocessingDrawing" xmlns:wp14="http://schemas.microsoft.com/office/word/2010/wordproces

In [5]:
document.namelist()

['[Content_Types].xml',
 '_rels/.rels',
 'word/_rels/document.xml.rels',
 'word/document.xml',
 'word/media/image1.emf',
 'word/theme/theme1.xml',
 'word/embeddings/oleObject3.bin',
 'word/media/image3.emf',
 'word/media/image2.emf',
 'word/embeddings/oleObject1.bin',
 'word/embeddings/oleObject2.bin',
 'word/settings.xml',
 'word/webSettings.xml',
 'word/fontTable.xml',
 'docProps/core.xml',
 'word/styles.xml',
 'docProps/app.xml']

In [6]:
r_bs = BeautifulSoup(document.read('word/_rels/document.xml.rels'), 'xml')
print(r_bs.prettify())

<?xml version="1.0" encoding="utf-8"?>
<Relationships xmlns:="http://schemas.openxmlformats.org/package/2006/relationships">
 <Relationship Id="rId8" Target="media/image3.emf" Type="http://schemas.openxmlformats.org/officeDocument/2006/relationships/image"/>
 <Relationship Id="rId3" Target="webSettings.xml" Type="http://schemas.openxmlformats.org/officeDocument/2006/relationships/webSettings"/>
 <Relationship Id="rId7" Target="embeddings/oleObject2.bin" Type="http://schemas.openxmlformats.org/officeDocument/2006/relationships/oleObject"/>
 <Relationship Id="rId2" Target="settings.xml" Type="http://schemas.openxmlformats.org/officeDocument/2006/relationships/settings"/>
 <Relationship Id="rId1" Target="styles.xml" Type="http://schemas.openxmlformats.org/officeDocument/2006/relationships/styles"/>
 <Relationship Id="rId6" Target="media/image2.emf" Type="http://schemas.openxmlformats.org/officeDocument/2006/relationships/image"/>
 <Relationship Id="rId11" Target="theme/theme1.xml" Type="h

https://olefile.readthedocs.io/en/latest/

In [7]:
f = document.open('word/embeddings/oleObject3.bin')
ole = olefile.OleFileIO(f)
ole.listdir()

[['\x01CompObj'], ['\x01Ole10Native'], ['\x03EPRINT'], ['\x03ObjInfo']]

In [8]:
ole.openstream('\x01Ole10Native').read()

b'\x81\x07\x00\x00\x02\x00lorem.txt\x00C:\\Users\\dronp\\Desktop\\lorem.txt\x00\x00\x00\x03\x00,\x00\x00\x00C:\\Users\\dronp\\AppData\\Local\\Temp\\lorem.txt\x00h\x06\x00\x00Adipisicing est non minim aute reprehenderit incididunt magna ad consectetur ad occaecat anim voluptate culpa fugiat excepteur enim qui nulla ut ea ullamco ut reprehenderit ad do veniam magna velit mollit incididunt amet pariatur nulla sit commodo dolor excepteur magna amet est anim deserunt amet et ad consectetur adipisicing magna nisi voluptate laboris sit laborum ut laborum aliqua sint dolor ullamco enim reprehenderit sunt et elit cillum est dolore consectetur ut labore exercitation ullamco anim ullamco ex ea tempor ad cillum ex eu magna dolor ullamco ad anim aliquip aute dolore ut sit excepteur enim reprehenderit exercitation dolore sit aliquip voluptate nulla cillum non reprehenderit officia labore pariatur pariatur anim in aliqua eu in anim deserunt dolor ex aliquip irure laboris consequat cupidatat id in dol

In [9]:
ole = olefile.OleFileIO(document.open('word/embeddings/oleObject2.bin'))
ole.listdir()

[['\x01CompObj'], ['\x01Ole10Native'], ['\x03EPRINT'], ['\x03ObjInfo']]

In [10]:
ole = olefile.OleFileIO(document.open('word/embeddings/oleObject1.bin'))
ole.listdir()

[['\x01CompObj'], ['\x01Ole'], ['\x03EPRINT'], ['\x03ObjInfo'], ['CONTENTS']]

https://docs.microsoft.com/en-us/openspecs/windows_protocols/ms-oleds/2677fcf2-ad48-4386-ba8f-b1b7baf4c02f

https://www.codeproject.com/Tips/784558/Extract-Embedded-Files-from-Microsoft-Office-Docum

In [11]:
import olefile
from zipfile import ZipFile

def extract_pdf(filename):
#     try:
        # Try to open the document as ZIP file
    with ZipFile(filename, "r") as document:
        entries = []
      # Find files in the word/embeddings folder of the ZIP file
        for entry in document.namelist():
            if not entry.startswith("word/embeddings/"):
                continue
            else:
                entries.append(entry)
        if not entries:
            return
        # Try to open the embedded OLE file
        for entry in entries:
            with document.open(entry) as f:
                if not olefile.isOleFile(f):
                    continue

                ole = olefile.OleFileIO(f)

                # CLSID for Adobe Acrobat Document
                if ole.root.clsid == "B801CA65-A1FC-11D0-85AD-444553540000":
                    if not ole.exists("CONTENTS"):
                        continue
                    # Extract the PDF from the OLE file
                    data = ole.openstream('CONTENTS').read()
                    # Does the embedded file have a %PDF- header?
                    if data[0:5] == b'%PDF-':
                        data_filename = "examples/out.pdf"
                else:
                    continue

                # Save the PDF
                with open(data_filename, "wb") as output_file:
                    output_file.write(data)

#     except:
#         print("Unable to open '%s'" % filename)

filename = 'examples/with_attachments.docx'
extract_pdf(filename)

In [12]:
with ZipFile(filename, "r") as document:
    entries = []
  # Find files in the word/embeddings folder of the ZIP file
    for entry in document.namelist():
        if not entry.startswith("word/embeddings/"):
            continue
        else:
            entries.append(entry)
    entry=entries[1] 
    f = document.open(entry)
    ole = olefile.OleFileIO(f)
  # CLSID for Adobe Acrobat Document
    if ole.root.clsid == "B801CA65-A1FC-11D0-85AD-444553540000":

      # Extract the PDF from the OLE file
        pdf_data = ole.openstream('CONTENTS').read()

        pdf_filename = "examples/out.pdf"

        # Save the PDF
        with open(pdf_filename, "wb") as output_file:
            output_file.write(pdf_data)